# ДЗ по ДЗ по "A/B-тестирование (семинары)"
## Семинар 9. Python применение продвинутых методов
## Задача
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.  
Mobile Games AB Testing with Cookie Cats


In [1]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
data = pd.read_csv('data/gb_sem_9_hw.csv')

In [3]:
data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [5]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [6]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

## Посмотрим сколько значений для А/В теста

In [7]:
data[data['version'] == 'gate_30'].shape

(44700, 5)

In [8]:
data[data['version'] == 'gate_40'].shape

(45489, 5)

## Проводим анализ на статистически значимые различия (стандартный)

In [9]:
continious_result(data[data['version'] == 'gate_30'], data[data['version'] == 'gate_40'], 'sum_gamerounds')

100%|██████████| 10000/10000 [00:11<00:00, 873.99it/s]


,effect_size,alpha,beta,CI,difference
sum_gamerounds,-0.005915,0.372228,0.856725,"[-0.962, 4.099]",1.160034


## Проводим анализ на статистически значимые различия (с бакетами)

In [10]:
for _ in range(100, 1001): 
    if data.shape[0] % _ == 0:
        print(f'Количество бакетов: {_}') 

Количество бакетов: 911


## Добавляем данные в бакеты

In [11]:
n_buckets = 911
data_2 = (data
 .sample(n=data.shape[0], replace=False)
 .reset_index(drop=True)
 .assign(bucket=list(range(n_buckets)) * int(data.shape[0] / n_buckets)))

In [12]:
data_2.head

<bound method NDFrame.head of         userid  version  sum_gamerounds  retention_1  retention_7  bucket
0      2247253  gate_40               6        False        False       0
1      1251901  gate_40             125         True        False       1
2      8730349  gate_30             131         True         True       2
3      8659794  gate_40              14         True        False       3
4      8270245  gate_30              10         True        False       4
...        ...      ...             ...          ...          ...     ...
90184  4293234  gate_30               0        False         True     906
90185  3124586  gate_40              10        False        False     907
90186  2333660  gate_30               4        False        False     908
90187  3784795  gate_30               1        False        False     909
90188   225892  gate_30              14        False        False     910

[90189 rows x 6 columns]>

## Подсчитываем среднее по бакетам

In [13]:
bucketed_data_2 = data_2.groupby(['version', 'bucket'])['sum_gamerounds'].agg(mu=np.mean).reset_index()
bucketed_data_2

,version,bucket,mu
0,gate_30,0,53.046512
1,gate_30,1,38.812500
2,gate_30,2,62.061224
3,gate_30,3,47.416667
4,gate_30,4,44.130435
...,...,...,...
1817,gate_40,906,43.239130
1818,gate_40,907,47.469388
1819,gate_40,908,44.250000
1820,gate_40,909,45.526316


In [14]:
# Сравним исходное выборочное среднее и среднее бакетных средних 
round(np.mean(data["sum_gamerounds"]), 5), round(np.mean(bucketed_data_2["mu"]), 5)

(51.87246, 51.87593)

## Вычисляем стат значимость по метрике "sum_gamerounds"

In [15]:
control_bucket = bucketed_data_2[bucketed_data_2.version == 'gate_30']
treatment_bucket = bucketed_data_2[bucketed_data_2.version == 'gate_40']
continious_result(control_bucket, treatment_bucket, 'mu')

100%|██████████| 10000/10000 [00:01<00:00, 9515.18it/s]


,effect_size,alpha,beta,CI,difference
mu,-0.04394,0.333638,0.844886,"[-0.909, 4.22]",1.275064


## Вывод: Стандартная функция посчитала в 11 раз медленнее чем с бакетами, результаты сходятся. Для метрики "sum_gamerounds" - Alpha > 5%, Beta>20%, доверительный интервал CI захватывает 0, можно было сказать, что статистически значимых различий нет между версиями, но следует заметить, что мощность теста power = 1-Beta, менее 20%. Мои рекомендации увеличить мощность теста до 80%. 

## Вычисляем стат значимость по метрике "retention_1"

In [16]:
proportion_result(data[data['version'] == 'gate_30'], data[data['version'] == 'gate_40'], 'retention_1')

100%|██████████| 10000/10000 [00:13<00:00, 764.54it/s]


,effect_size,alpha,beta,CI,difference
retention_1,-0.003823,0.560444,0.911819,"[-0.009, 0.005]",-0.001959


## Вывод: Статистически значимых различий нет, но мощность теста маловата

## Вычисляем стат значимость по метрике "retention_7"

In [17]:
proportion_result(data[data['version'] == 'gate_30'], data[data['version'] == 'gate_40'], 'retention_7')

100%|██████████| 10000/10000 [00:09<00:00, 1051.37it/s]


,effect_size,alpha,beta,CI,difference
retention_7,0.012776,0.056683,0.519844,"[-0.0, 0.01]",0.004984


## Вывод: Прям на грани, если еще немного поднять мощность теста, может покажет статистически значимые различия